### Importante!

Este notebook foi executado usando Databricks com uma versão 12.2 LTS Runtime!

O objetivo é mostrar as **consequências e diferenças** entre a clonagem de tabelas Delta usando **Shallow** e Deep Clones enquanto não estiver sob a cobertura do **Unity Catalog**, provando que o elemento de cópia zero do Shallow Clone irá **falhar* * em caso de exclusão (com Vacuum) na fonte de dados.

Insira `spark.databricks.delta.retentionDurationCheck.enabled false` durante a criação das configurações do cluster para poder usar o VACUUM corretamente neste notebook.

O objetivo deste notebook é apenas mostrar a versão dos comandos SQL para Python, **use sempre a versão SQL como referência**, pois foi a usada durante o curso SQL do Databricks.

In [0]:
%fs ls /databricks-datasets/COVID/coronavirusdataset

In [0]:
df_covid = spark.read\
  .option("header","true")\
  .option("inferSchema","true")\
  .option('delimiter', ',')\
  .csv("dbfs:/databricks-datasets/COVID/coronavirusdataset/PatientInfo.csv")

df_covid.write.format("delta").mode("overwrite").saveAsTable("COVID_Clone")

In [0]:
df_covid_clone = spark.read.table("COVID_Clone")
display(df_covid_clone.limit(5))

In [0]:
df_covid_clone.createOrReplaceTempView("tempView")
df_clone_covid_deep = spark.sql("SELECT * FROM tempView")

In [0]:
display(df_covid_clone.history())

In [0]:
df_clone_covid_shallow = df_covid_clone.clone('dbfs://hive_metastore_test/covid_shallow_clone', isShallow = True)

In [0]:
display(df_covid_clone.count())

In [0]:
display(df_clone_covid_deep.count())

In [0]:
display(df_clone_covid_shallow.count())

In [0]:
df_covid_clone.delete("sex = 'male'")

In [0]:
display(df_covid_clone.count())

In [0]:
display(df_covid_clone.history())

In [0]:
display(df_clone_covid_deep.count())

In [0]:
display(df_clone_covid_shallow.filter(col(sex) = 'male'))

In [0]:
df_covid_clone.vacuum(0)

In [0]:
display(df_clone_covid_shallow.filter(col(sex) = 'male'))

In [0]:
display(df_clone_covid_deep.filter(col(sex) = 'male'))